In [ ]:
import torch
import numpy as np

from prep import TimeWindowTransformer, LabelWindowExtractor

from regressors import NNRegressor
from loss_functions import RMSELoss, NMSELoss
from models import ConvNN, EMGConvNet, EMGConvNet2D

from validation import RMSE, NMSE

### Loading, windowing, splitting

In [ ]:
PATH = f'/Users/marco/PROJECTS/data/'
# PATH = r'C:\Users\gianm\Documents\Uni\Big Data\F422\project\data\\'
DATASET = 'freemoves' # change this to guided/freemoves if needed

X = np.load(PATH + f'{DATASET}/{DATASET}_dataset_X.npy')
Y = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')

tw_extractor = TimeWindowTransformer(size = 500, step = 100)
label_extractor = LabelWindowExtractor(size = 500, step = 100)

X_windows = tw_extractor.transform(X)
Y_labels = label_extractor.transform(Y)

train_val_idx = [0,1,2,3]
test_idx = 4

X_train_val = X_windows[train_val_idx].reshape(-1, *X_windows.shape[2:])
Y_train_val = Y_labels[train_val_idx].reshape(-1, *Y_labels.shape[2:])

X_test = X_windows[test_idx]
Y_test = Y_labels[test_idx]

print(X_train_val.shape)
print(Y_train_val.shape)
print(X_test.shape)
print(Y_test.shape)

### Testing Marco's model

In [ ]:
my_NNRegressor = NNRegressor(
    model_class=ConvNN,
    model_parameters={'end_dim': 51},
    loss_fnc=RMSELoss(),
    batch_size = 50,
    learning_rate = 0.001,
    max_epochs = 50,
    patience = 10
)

my_NNRegressor.fit_with_validation(X_train_val, Y_train_val, X_test, Y_test)

In [ ]:
# test on unseen data
Y_test_pred = my_NNRegressor.predict(X_test)

print(RMSE(Y_test_pred, Y_test))
print(NMSE(Y_test_pred, Y_test))

### Testing Thomas' models

In [ ]:
model_config = {
    "conv_layers_config": [(16, 5, 1), (32, 5, 2)],
    "fc_layers_config": [512, 256, 128, 64],
    "output_dim": 51,
    "verbose": False
}

my_NNRegressor = NNRegressor(
    model_class=EMGConvNet,
    model_parameters=model_config,
    loss_fnc=RMSELoss(),
    batch_size = 50,
    learning_rate = 0.001,
    max_epochs = 50,
    patience = 10
)

# this shows the validation directly during training to gauge overfit/underfit
my_NNRegressor.fit_with_validation(X_train_val, Y_train_val)